In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import root_mean_squared_error
import plotly.graph_objects as go

### Es 1
Hai a disposizione un file `data.csv` contenente dati mensili di passeggeri con due colonne:

- `date`: data in formato `YYYY-MM` (mese/anno)
- `passengers`: numero di passeggeri per quel mese


Costruisci un modello di **regressione polinomiale** che approssima l’andamento del numero di passeggeri nel tempo.

1. Carica il dataset.
2. Convertilo in un formato numerico utilizzando una colonna `mese_numerico` che conti i mesi a partire da gennaio 1949.
3. Applica una regressione polinomiale (grado a tua scelta).
4. Calcola l’RMSE tra i valori reali e quelli predetti.
5. Visualizza i dati reali e la curva stimata con Plotly.

In [2]:
# carico il dataset
df = pd.read_csv('data.csv')

# converto in formato numerico
df['date'] = pd.to_datetime(df['date'], format='%Y-%m')
df['mese_numerico'] = (df['date'].dt.year - 1949) * 12 + (df['date'].dt.month - 1)
df = df.dropna(subset=['passengers'])

# regressione polinomiale di grado 2
x = df['mese_numerico'].values
y = df['passengers'].values

coeffs = np.polyfit(x, y, deg=2)
poly = np.poly1d(coeffs)
y_pred = poly(x)

# calcolo RMSE tra valori reali e predetti
rmse = root_mean_squared_error(y, y_pred)
print(f'RMSE = {rmse:.2f}')

# plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df['date'], 
    y=y, 
    mode='markers',
    name='Dati reali', 
    marker=dict(color='green')
))

fig.add_trace(go.Scatter(
    x=df['date'],
    y=y_pred,
    mode='lines',
    name='Fit polinomiale',
    line=dict(color='blue')
))

fig.update_layout(
    title='Regressione polinomiale sul numero di passeggeri',
    xaxis_title='Date',
    yaxis_title='Numero di passeggeri',
    template='ggplot2'
)

fig.show()

RMSE = 44.45


### Es2. 
Costruisci una web app con Dash che permette all’utente di scegliere il grado del polinomio per adattare un modello di regressione ai dati non lineari e vedere il risultato aggiornarsi dinamicamente.


1. Genera 100 punti x tra -3 e 3.

2. Calcola ad esempio y = x³ - x + rumore.

3. Costruisci un'interfaccia Dash con:
    - uno slider per scegliere il grado del polinomio (1–10),
    - un grafico Plotly che mostra i dati e la curva stimata.

4. Usa PolynomialFeatures + LinearRegression da scikit-learn per stimare la curva

In [3]:
import dash
from dash.dependencies import Input, Output
from dash import dcc, html
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [ ]:
# genera 100 punti x tra -3 e 3
def generate(n=100, noise_std=1.0, seed=0):
    np.random.seed(seed)
    x = np.linspace(-3,3,n)
    y = x**3 - x + np.random.normal(0, noise_std, size=n)
    return x,y

x, y = generate()

# creazione app
app = dash.Dash(__name__)
server = app.server

app.layout = html.Div([
    html.H1('Regressione polinomiale'),
    
    dcc.Slider(
        id='degree-slider',
        min=1,
        max=10,
        step=1,
        value=3,
        marks={i: str(i) for i in range(1,11)},
        tooltip={'always_visible':False, 'placement':'bottom'}
    ),

    dcc.Graph(
        id='poly-graph'
    ),

    html.Div(
        id='output-degree',
        style={'marginTop': 20, 'fontSize': 19}
    ),

],
style={'widht':'80%', 'margin': 'auto'}
)

@app.callback(
    [Output('poly-graph', 'figure'), Output('output-degree', 'children')],
    [Input('degree-slider', 'value')]
)

def update_graph(degree):
    poly = PolynomialFeatures(degree=degree, include_bias=True)
    X_poly = poly.fit_transform(x.reshape(-1,1))

    # regressione
    lm = LinearRegression(fit_intercept=False)
    lm.fit(X_poly, y)

    # predizione
    x_fit = np.linspace(x.min(), x.max(), 200)
    X_fit_poly = poly.transform(x_fit.reshape(-1,1))
    y_fit = lm.predict(X_fit_poly)

    # creo figura
    trace_data = go.Scatter(
        x=x, 
        y=y, 
        mode='markers', 
        name='Dati'
    )

    trace_fit = go.Scatter(
        x=x_fit,
        y=y_fit,
        mode='lines',
        name=f'fit di grado {degree}'
    )

    fig = go.Figure(data=[trace_data, trace_fit])

    fig.update_layout(
        title=f'Regressione polinomiale di grado {degree}',
        xaxis_title='X',
        yaxis_title='Y',
        template='ggplot2'
    )

    return fig, f'Degree: {degree}'

if __name__ == '__main__':
    app.run(debug=True)